In [9]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------  16.3/16.5 MB 84.9 MB/s eta 0:00:01
   ---------------------------------------- 16.5/16.5 MB 74.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import mysql.connector

In [2]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Saurav@123",
    database="jobs_data",
    port=3306
)

print("Connection Successful")

Connection Successful


In [3]:
query = "SELECT * FROM job_postings2 LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

C:\Users\Varun\AppData\Local\Temp\ipykernel_19608\2538642093.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,job_id,company_id,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country
0,0,0,Data Analyst,Marketing Data Analyst,Anywhere,via LinkedIn,Full-time,TRUE,Serbia,9/25/2023 17:46,FALSE,FALSE,Serbia
1,55,1,Cloud Engineer,Storage and Virtualization Engineer,"Kuwait City, Kuwait",via Trabajo.org,Full-time,FALSE,Kuwait,7/30/2023 17:49,TRUE,FALSE,Kuwait
2,66,2,Data Analyst,Data Analyst et Scientist F/H,"Paris, France",via Emplois Trabajo.org,Full-time,FALSE,France,7/28/2023 17:28,FALSE,FALSE,France
3,76,3,Data Engineer,Data Engineer,"Denver, CO",via LinkedIn,Contractor,FALSE,"Illinois, United States",4/3/2023 17:14,FALSE,FALSE,United States
4,81,4,Data Engineer,Data Engineer,Anywhere,via LinkedIn,Contractor,TRUE,Canada,3/25/2023 17:25,FALSE,FALSE,Canada


In [5]:
# Tables_name 
tables_name = ['company_dim', 'job_postings2', 'skills_job_dim2', 'skills_dim2']

# empty_dict to store df's
dataframes = {}

for table_name in tables_name:
    query = f"SELECT * FROM {table_name}"
    df    = pd.read_sql(query,conn)

    dataframes[table_name] = df

conn.close()

C:\Users\Varun\AppData\Local\Temp\ipykernel_19608\3936254100.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df    = pd.read_sql(query,conn)


In [6]:
# Loading the dataframes 
company         = dataframes["company_dim"]
job_postings2   = dataframes["job_postings2"]
skills_job_dim2 = dataframes["skills_job_dim2"]
skills_dim2     = dataframes["skills_dim2"]

In [14]:
# Merge all the tables as per requirements
merge = pd.merge(company, job_postings2, on='company_id', how='left')
merge.shape

(787686, 17)

In [15]:
# merge skills_jobdim2 on job_id
merge = pd.merge(merge, skills_job_dim2, on='job_id', how='left')
merge.shape

(1434225, 18)

In [16]:
# merge skill_dim2 on skill_id
merge = pd.merge(merge, skills_dim2, on='skill_id', how='left')
merge.shape

(1434225, 20)

In [17]:
# list of column swe have 
merge.columns

Index(['company_id', 'name', 'link', 'link_google', 'thumbnail', 'job_id',
       'job_title_short', 'job_title', 'job_location', 'job_via',
       'job_schedule_type', 'job_work_from_home', 'search_location',
       'job_posted_date', 'job_no_degree_mention', 'job_health_insurance',
       'job_country', 'skill_id', 'skills', 'type'],
      dtype='object')

In [18]:
# remove unneccessary columns
col_list = ['link', 'link_google', 'thumbnail',
            'job_via','job_no_degree_mention', 'job_health_insurance',
            'job_country','skill_id','type']
merge.drop(columns=col_list,inplace=True)

In [19]:
merge.shape

(1434225, 11)

In [25]:
# filling na in skills column
merge['skills'] = merge['skills'].fillna('').astype(str)

In [28]:
# list of remaining columns
merge.columns

Index(['company_id', 'name', 'job_id', 'job_title_short', 'job_title',
       'job_location', 'job_schedule_type', 'job_work_from_home',
       'search_location', 'job_posted_date', 'skills'],
      dtype='object')

In [30]:
# grouping by column 
final_df = merge.groupby(['company_id', 'name', 'job_id', 'job_title_short', 'job_title',
       'job_location', 'job_schedule_type', 'job_work_from_home',
       'search_location', 'job_posted_date'])['skills'].agg(','.join).reset_index()

In [31]:
final_df

,company_id,name,job_id,job_title_short,job_title,job_location,job_schedule_type,job_work_from_home,search_location,job_posted_date,skills
0,0,Cryptology,0,Data Analyst,Marketing Data Analyst,Anywhere,Full-time,TRUE,Serbia,9/25/2023 17:46,"sql,python"
1,0,Cryptology,923232,Data Analyst,Marketing Data Analyst,Anywhere,Full-time,TRUE,Spain,9/25/2023 17:51,"sql,python"
2,0,Cryptology,1695656,Data Analyst,Marketing Data Analyst,Anywhere,Full-time,TRUE,United Kingdom,9/25/2023 18:11,
3,0,Cryptology,1819517,Data Analyst,Marketing Data Analyst,"Madrid, Spain",Full-time,FALSE,Spain,10/10/2023 10:32,"sql,python"
4,1,Edraak,55,Cloud Engineer,Storage and Virtualization Engineer,"Kuwait City, Kuwait",Full-time,FALSE,Kuwait,7/30/2023 17:49,
...,...,...,...,...,...,...,...,...,...,...,...
787681,787652,Rotary Engineering Pte. Ltd.,1424742,Senior Data Scientist,Senior Design Engineer (Instrumentation),Singapura,Kontraktor,FALSE,Singapore,3/13/2023 6:16,
787682,787674,Pure App,549918,Senior Data Engineer,Senior Data Engineer,"Berlin, Jerman",Pekerjaan tetap,FALSE,Germany,3/13/2023 6:19,
787683,787675,Staff-UP Consulting Group,782857,Business Analyst,PreSales Engineer,"Almaty, Kazakhstan",Pekerjaan tetap,FALSE,Kazakhstan,3/12/2023 6:31,
787684,787680,CAREERSTAR INTERNATIONAL PTE. LTD.,828965,Software Engineer,DevOps Engineer,Singapura,Pekerjaan tetap,FALSE,Singapore,3/13/2023 6:16,
